<a href="https://colab.research.google.com/github/Hitika-Jain/ParkInsight/blob/main/italianlstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install tensorflow keras librosa numpy pandas scikit-learn matplotlib

In [ ]:
import librosa

def load_audio(file_path, sr=16000):
    audio, _ = librosa.load(file_path, sr=sr)
    return audio

In [ ]:
def extract_spectrogram(audio, sr=16000, n_mels=128, n_fft=2048, hop_length=512):
    spectrogram = librosa.feature.melspectrogram(
        y=audio, sr=sr, n_mels=n_mels, n_fft=n_fft, hop_length=hop_length
    )
    log_spectrogram = librosa.power_to_db(spectrogram, ref=np.max)
    return log_spectrogram

In [ ]:
from sklearn.preprocessing import StandardScaler

def normalize_spectrogram(spectrogram):
    scaler = StandardScaler()
    normalized = scaler.fit_transform(spectrogram)
    return normalized

In [ ]:
def create_sequences(spectrogram, seq_length=100, hop_length=50):
    sequences = []
    for i in range(0, spectrogram.shape[1] - seq_length, hop_length):
        sequences.append(spectrogram[:, i:i+seq_length])
    return np.array(sequences)

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, LSTM, Dense, Flatten, Reshape

def build_neurospeech_model(input_shape, num_classes):
    inputs = Input(shape=input_shape)

    # CNN layers for feature extraction
    x = Conv2D(32, (3, 3), activation='relu')(inputs)
    x = MaxPooling2D((2, 2))(x)
    x = Conv2D(64, (3, 3), activation='relu')(x)
    x = MaxPooling2D((2, 2))(x)
    x = Flatten()(x)

    # Reshape for LSTM
    x = Reshape((-1, 64))(x)

    # LSTM layers
    x = LSTM(128, return_sequences=True)(x)
    x = LSTM(64)(x)

    # Output layer
    outputs = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs, outputs)
    return model

In [ ]:
model = build_neurospeech_model(input_shape=(128, 100, 1), num_classes=2)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
from tensorflow.keras.utils import to_categorical
import numpy as np
# Assuming X_train and y_train are your preprocessed data and labels
X_train = np.expand_dims(X_train, axis=-1)  # Add channel dimension
y_train = to_categorical(y_train, num_classes=2)  # Convert labels to one-hot encoding

# Train the model
history = model.fit(X_train, y_train, batch_size=32, epochs=20, validation_split=0.2)

NameError: name 'X_train' is not defined